In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import re
import pandas as pd
import numpy as np
from collections import defaultdict


init_notebook_mode(connected=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df = pd.read_csv('movies_list.csv', index_col=0)
df.head()

,countries,genres,movie_duration,year
0,['США'],"['мультфильм', 'комедия', 'приключения', 'семе...",89,2008
1,['Россия'],"['драма', 'история']",124,2008
2,"['США', 'Германия', 'Китай']","['ужасы', 'фэнтези', 'боевик', 'триллер', 'при...",112,2008
3,['Россия'],['комедия'],106,2008
4,"['США', 'Германия']","['фэнтези', 'боевик', 'триллер', 'криминал']",110,2008


In [2]:
def parse_list(lst_str):
    return list(map(lambda x: x.strip(), re.sub(r'[\'\[\]]', '', lst_str).split(','))) 

In [3]:
#determine the unique genres
genres = set()
for m in df.genres:
    genres.update(g for g in parse_list(m))
genres = sorted(genres)

for genre in genres:
    df[genre] = [genre in movie for movie in df.genres]

df_genres = pd.DataFrame(df.groupby(['year'])[genres].sum())
temp = np.argsort(-df_genres.values, axis=1)[:, :3]
result = pd.DataFrame(df_genres.columns[temp], 
                      columns=['{}'.format(i) for i in range(1, 4)],
                      index=df_genres.index)

pd.DataFrame(result)

,1,2,3
year,,,
2008,приключения,боевик,комедия
2009,приключения,боевик,драма
2010,приключения,фэнтези,боевик
2011,приключения,боевик,комедия
2012,приключения,боевик,фантастика
2013,приключения,боевик,фантастика
2014,приключения,боевик,фантастика
2015,приключения,боевик,фантастика
2016,приключения,фэнтези,семейный


In [4]:
df['genres'] = df['genres'].fillna('[]')
genres_data = []
for record in df.to_dict(orient = 'records'):
    genres_lst = parse_list(record['genres'])
    for genre in genres_lst:
        copy = record.copy()
        copy['genre'] = genre
        copy['weight'] = 1./len(genres_lst)
        genres_data.append(copy)

genres_df = pd.DataFrame.from_dict(genres_data)
genres_coincidents = {}

for item in df.genres:
    parsed_genres = parse_list(item)
    for genre1 in parsed_genres:
        if genre1 not in genres_coincidents:
            genres_coincidents[genre1] = defaultdict(int)
        for genre2 in parsed_genres:
            genres_coincidents[genre1][genre2] += 1

genres_coincidents_df = pd.DataFrame.from_dict(genres_coincidents).fillna(0)

# отнормируем таблицу на количество фильмов каждого жанра
genres_coincidents_df_norm = genres_coincidents_df\
    .apply(lambda x: x/genres_df.groupby('genre').year.count(), axis = 1)

heatmap = go.Heatmap(
    z = genres_coincidents_df_norm.values,
    x = genres_coincidents_df_norm.index.values,
    y = genres_coincidents_df_norm.columns
)
layout = go.Layout(
    title = 'Связанные жанры'
)

fig = go.Figure(data = [heatmap], layout = layout)
iplot(fig)

In [5]:
#determine the unique countries
countries = set()
for m in df.countries:
    countries.update(g for g in parse_list(m))
countries = sorted(countries)

for country in countries:
    df[country] = [country in movie for movie in df.countries]

df_countries = pd.DataFrame(df.groupby(['year'])[countries].sum())
temp = np.argsort(-df_countries.values, axis=1)[:, :3]
result = pd.DataFrame(df_countries.columns[temp], 
                      columns=['{}'.format(i) for i in range(1, 4)],
                      index=df_countries.index)

pd.DataFrame(result)


,1,2,3
year,,,
2008,США,Россия,Германия
2009,США,Россия,Великобритания
2010,США,Россия,Великобритания
2011,США,Россия,Великобритания
2012,США,Великобритания,Германия
2013,США,Россия,Франция
2014,США,Великобритания,Канада
2015,США,Великобритания,Россия
2016,США,Великобритания,Россия


In [26]:
country_count = df[countries].sum().sort_values(ascending = False).head(5)
pd.DataFrame({'Топ-5 стран': country_count})

,Топ-5 стран
США,165
Россия,33
Великобритания,31
Канада,12
Германия,12


In [27]:
genre_count = df[genres].sum().sort_values(ascending = False).head(5)
pd.DataFrame({'Топ-5 жанров': genre_count})

,Топ-5 жанров
приключения,133
боевик,100
комедия,78
фэнтези,73
фантастика,73


In [6]:
dyn_year_df = pd.DataFrame(df.groupby(['year'],as_index = False)['Россия',
                                                                 'США',
                                                                 'Великобритания',
                                                                 'Канада',
                                                                 'Германия'].sum())
dyn_year_df.rename(columns={'США': 'Usa',
                            'Россия': 'Rus',
                            'Великобритания': 'Gb',
                            'Канада': 'Can',
                            'Германия': 'Ger' }, inplace=True)
trace_rus = go.Scatter(
    x = dyn_year_df.year,
    y = dyn_year_df.Rus,
    mode = 'lines',
    name = u'Россия'
)
trace_usa = go.Scatter(
    x = dyn_year_df.year,
    y = dyn_year_df.Usa,
    mode = 'lines',
    name = u'США'
)
trace_gb = go.Scatter(
    x = dyn_year_df.year,
    y = dyn_year_df.Gb,
    mode = 'lines',
    name = u'Великобритания'
)
trace_can = go.Scatter(
    x = dyn_year_df.year,
    y = dyn_year_df.Can,
    mode = 'lines',
    name = u'Канада'
)
trace_ger = go.Scatter(
    x = dyn_year_df.year,
    y = dyn_year_df.Ger,
    mode = 'lines',
    name = u'Германия'
)
layout = go.Layout(
    title='Динамика',
)   

fig = go.Figure(data = [trace_usa, trace_rus, trace_gb, trace_can, trace_ger], layout = layout)
iplot(fig)

In [21]:
df['country'] = [parse_list(country)[0] for country in df.countries] 
def get_region(row):
    regions = [{'region_name': 'RU', 'countries': ['Россия']},
               {'region_name': 'ASIA', 'countries': ['Китай','Япония','Тайвань','Гонконг','Сингапур','Корея Южная','Индия']},
               {'region_name': 'EU', 'countries': ['Великобритания','Германия','Франция','Италия','Испания','Чехия','Мальта','Польша','Украина','Исландия','Словения','Бельгия']},
               {'region_name': 'NA', 'countries': ['США','Канада']},
               {'region_name': 'Other', 'countries': ['Новая Зеландия','Австралия']}]
    for item in regions:
        if row['country'] in item['countries']:
            return item['region_name']

df['region'] = df.apply (lambda row: get_region (row),axis=1)
test = pd.crosstab(df.year, df.region)
test.reset_index(level=0, inplace=True)
trace1 = go.Bar(
    x=test.year,
    y=test.ASIA,
    name='ASIA'
)
trace2 = go.Bar(
    x=test.year,
    y=test.EU,
    name='EU'
)
trace3 = go.Bar(
    x=test.year,
    y=test.NA,
    name='NA'
)
trace4 = go.Bar(
    x=test.year,
    y=test.Other,
    name='Other'
)
trace5 = go.Bar(
    x=test.year,
    y=test.RU,
    name='Ru'
)
data = [trace1, trace2, trace3, trace4,trace5]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)